# Training The Transfer Learning Models
## VGG16 based transfer learning technique for comparison with SimpleNet
The notebook contains code used for training a transfer learning method using VGG16 model. 
The model classifies preprocessed retinal OCT scans into three categories: urgent referral, routine referral
and normal. **5-fold cross validation** was used to present an accurate estimate of the efficacy of the model 
on unseen data. The notebooks were run on the kaggle platform for training purposes. Since the 5-folds
and initial weights of the deep neural net are random, the accuracies might not match with the accuracies
given in the publication.

### Table of Contents
* [1. Importing all modules](###-1.-Importing-all-modules)
* [2. Loading train and test data](###-2.-Loading-train-and-test-data)
* [3. Generator Classes using Sequence class](###-3.-Generator-Classes-using-Sequence-class)
* [4. Functions for building and loading the transfer learning model](###-4.-Functions-for-building-and-loading-the-transfer-learning-model)
* [5. Functions for training and cross-validation](###-5.-Functions-for-training-and-cross-validation)
* [6. Training the model](###-6.-Training-the-model)
* [7. Performance metrics of VGG16 based model](###-7.-Performance-metrics-of-VGG16-based-model)
* [8. Performance of model on test set](###-8.-Performance-of-model-on-test-set)

### 1. Importing all modules

In [1]:
import os
import keras
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.utils import Sequence
from cv2 import * #Import functions from OpenCV
import cv2
import glob
from skimage.transform import resize
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from keras.models import model_from_json
import json
from statistics import mean



Using TensorFlow backend.


### 2. Loading train and test data

Training data is split randomly into 5 folds.

**Variable description**
- img: corresponds to training images
- y: corresponds to training images categories

In [2]:
def load_train_data():
    img = glob.glob("../input/preprocessedoctcropped224/preprocessed_oct_cropped/preprocessed_OCT_cropped/train/CNV/*.jpeg");
    img = img + glob.glob("../input/preprocessedoctcropped224/preprocessed_oct_cropped/preprocessed_OCT_cropped/train/DME/*.jpeg");
    l = len(img);
    y = np.zeros((l,3))
    y[:,2] =1
    img = img + glob.glob("../input/preprocessedoctcropped224/preprocessed_oct_cropped/preprocessed_OCT_cropped/train/DRUSEN/*.jpeg");
    m = len(img);
    k = np.zeros((m-l,3));
    k[:,1] = 1;
    y = np.append(y,k, axis =0);
    img = img + glob.glob("../input/preprocessedoctcropped224/preprocessed_oct_cropped/preprocessed_OCT_cropped/train/NORMAL/*.jpeg");
    k = np.zeros((len(img)-m,3));
    k[:,0] = 1;
    y = np.append(y,k, axis =0);
    from sklearn.utils import shuffle
    img, y = shuffle(img,y)
    img, y = shuffle(img,y)
    img, y = shuffle(img,y)

def kfold_on_train():
    from sklearn.model_selection import KFold
    ## Training with K-fold cross validation
    kf = KFold(n_splits=5, random_state=None, shuffle=True)
    kf.get_n_splits(img)

    X = np.array(img)
    y = np.array(y)
    np.savetxt('X.out',X,fmt='%s')
    np.savetxt('y.out',y)
    i=1
    for train_index, test_index in kf.split(img):
        trainData = X[train_index]
        testData = X[test_index]
        trainLabels = y[train_index]
        testLabels = y[test_index]
        np.savetxt('train_index-'+str(i)+'.out',train_index)
        np.savetxt('test_index-'+str(i)+'.out',test_index)
        i+=1

#reading test images, because can't use validation generator here, as that limits the time
def load_test_data():
    test_img = []
    test_y=[]
    for images in glob.glob("../input/preprocessedoctcropped224/preprocessed_oct_cropped/preprocessed_OCT_cropped/test/CNV/*.jpeg"):
        n = resize(imread(images), (224, 224,3))
        test_img.append(n)
        test_y.append(2)    

    for images in glob.glob("../input/preprocessedoctcropped224/preprocessed_oct_cropped/preprocessed_OCT_cropped/test/DME/*.jpeg"):
        n = resize(imread(images), (224, 224,3))
        test_img.append(n)
        test_y.append(2)    

    for images in glob.glob("../input/preprocessedoctcropped224/preprocessed_oct_cropped/preprocessed_OCT_cropped/test/DRUSEN/*.jpeg"):
        n = resize(imread(images), (224, 224,3))
        test_img.append(n)
        test_y.append(1)    

    for images in glob.glob("../input/preprocessedoctcropped224/preprocessed_oct_cropped/preprocessed_OCT_cropped/test/NORMAL/*.jpeg"):
        n = resize(imread(images), (224, 224,3))
        test_img.append(n)
        test_y.append(0)    

    test_img = np.asarray(test_img)
    test_y = np.asarray(test_y)


### 3. Generator Classes using Sequence class
Since our training dataset is large, we would use the Sequence class to help us feed images in batches for training.
Different generator classes are defined for training on four folds and cross-validation on the remaining fold.

In [3]:
class Training_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.n = len(image_filenames)

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx_min = idx*self.batch_size
        idx_max = np.amin([((idx+1)*self.batch_size),self.n])
        batch_x = self.image_filenames[idx_min:idx_max]
        batch_y = self.labels[idx_min:idx_max]
        X = np.array([
            resize(imread(file_name), (224, 224,3))
               for file_name in batch_x])
        y = np.array(batch_y)
        return X,y
    
class Validation_Generator(Sequence):

    def __init__(self, image_filenames, batch_size):
        self.image_filenames = image_filenames
        self.batch_size = batch_size
        self.n = len(image_filenames)

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx_min = idx*self.batch_size
        idx_max = np.amin([((idx+1)*self.batch_size),self.n])
        batch_x = self.image_filenames[idx_min:idx_max]
        X = np.array([
            resize(imread(file_name), (224, 224,3))
               for file_name in batch_x])
        return X
    

### 4. Functions for building and loading the transfer learning model

For each cross-validation build_model() builds the VGG16 based model whereas load_ith_model() 
loads the ith cross-validation model from memory for testing and validation. 

In [ ]:
def build_model():
    #load vgg16 without dense layer and with theano dim ordering
    base_model = VGG16(weights='../input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top = False, input_shape = (224,224,3))
    for layer in base_model.layers:
        layer.trainable = False                                                                                                                                        
    num_classes = 3

    x = Flatten()(base_model.output)
    predictions = Dense(num_classes, activation = 'softmax')(x)

    #create graph of your new model
    model = Model(input = base_model.input, output = predictions)

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def load_ith_model(i):
    json_file = open('model-'+str(i)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights('model-'+str(i)+'.h5')

    #compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


### 5. Functions for training and cross-validation
For each cross-validation i, train_save_validate() trains the model, saves model-i and saves the validation performance
metrics on validation fold. perfromance_metrics() derives all performance measures from the confusion matrix whereas
 print_performance() prints the validation metrics in a readable format.

In [ ]:
def train_save_validate(i):
    batch_size = 64
    train_index = np.genfromtxt('train_index-'+str(i)+'.out').astype(int)
    test_index = np.genfromtxt('test_index-'+str(i)+'.out').astype(int)
    trainData = X[train_index]
    testData = X[test_index]
    trainLabels = y[train_index]
    testLabels = y[test_index]

    trainGenerator = Training_Generator(trainData,trainLabels,batch_size)
    valGenerator = Validation_Generator(testData,400)
    
    model = build_model()
    if i ==1:
        model.summary()
        
    model.fit_generator(
            generator = trainGenerator,
            steps_per_epoch=(len(trainData)//batch_size),
            epochs=4, verbose = 2, class_weight = [0.70596402, 4.19022748, 0.74357918])
    
    model_json = model.to_json()
    with open('model-'+str(i)+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('model-'+str(i)+'.h5')
    
    test_y = np.argmax(testLabels,axis=1)    
    ans = np.argmax(model.predict_generator(valGenerator),axis =1)
    
    # performance metrics  
    return performance_metrics(test_y,ans)

def performance_metrics(test_y,ans):
    acc = accuracy_score(test_y,ans)
    print('Accuracy = ',acc)
    target_names = ['CLass Normal', 'CLass Early', 'Class Late']
    cm = confusion_matrix(test_y, ans) 
    sens0 = cm[0,0]/(cm[0,0]+cm[0,1]+cm[0,2])
    spec0 = (cm[1,1]+cm[1,2]+cm[2,1] +cm[2,2])/((cm[1,1]+cm[1,2]+cm[2,1] +cm[2,2])+(cm[1,0]+cm[2,0]))

    sens1 = cm[1,1]/(cm[1,0]+cm[1,1]+cm[1,2])
    spec1 = (cm[0,0]+cm[2,0]+cm[0,2] +cm[2,2])/((cm[0,0]+cm[2,0]+cm[0,2] +cm[2,2])+(cm[0,1]+cm[2,1]))

    sens2 = cm[2,2]/(cm[2,0]+cm[2,1]+cm[2,2])
    spec2 = (cm[0,0]+cm[0,1]+cm[1,0] +cm[1,1])/((cm[0,0]+cm[0,1]+cm[1,0] +cm[1,1])+(cm[0,2]+cm[1,2]))
  
    rep=classification_report(test_y, ans, target_names=target_names, digits = 4, labels=range(0,3),output_dict=True)
    return [acc,sens0,spec0,sens1,spec1,sens2,spec2],cm,rep

def print_performance(p):
    a = list(p[1][2].keys())
    b = list(p[1][2][a[0]].keys())


    for i in range (0,1):
        s = np.zeros(7)
        pn = 0
        pe = 0
        pl =0
        fn =0
        fe = 0
        fl = 0
        rn = 0
        re = 0
        rl = 0
        for j in range(1,6):
            s = s + p[j][0]
            pn = pn +  p[j][2][a[0]][b[0]]
            pe = pe +  p[j][2][a[1]][b[0]]
            pl = pl +  p[j][2][a[2]][b[0]]
            fn = fn + p[j][2][a[0]][b[2]]
            fe = fe + p[j][2][a[1]][b[2]]
            fl = fl +  p[j][2][a[2]][b[2]]
            rn = rn +  p[j][2][a[0]][b[1]]
            re = re +  p[j][2][a[1]][b[1]]
            rl = rl +  p[j][2][a[2]][b[1]]

        print('\nacc = ',round(s[0]*20,2))
        print("              sensitivity  specificity      precision      f1-score")
        print('\nClass Normal: ',round(s[1]*20,2), ",       ", round(s[2]*20,2), ",       ", round(pn*20,2),",       ", round(fn*20,2))
        print('\nClass Early:  ',round(s[3]*20,2), ",       ", round(s[4]*20,2), ",       ", round(pe*20,2),",       ", round(fe*20,2))
        print('\nClass Late:   ',round(s[5]*20,2), ",       ", round(s[6]*20,2), ",       ", round(pl*20,2),",       ", round(fl*20,2))

        print('\nAverage')
        print('\nsensitivity: ',round(mean([s[1],s[3],s[5]])*20,2), ", specificity: ",round(mean([s[2],s[4],s[6]])*20,2),
              ", precision: ", round(mean([pn,pe,pl])*20,2),", f1-score: ", round(mean([fn,fe,fl])*20,2))


        

### 6. Training the model

In [ ]:
load_train_data()
kfold_on_train()
load_test_data()

valid_perf = {}

for i in range(1,5):
    print("\n====== K Fold Validation step => %d/%d =======" % (i,5))
    
    valid_perf[i]= train_save_validate(i)

### 7. Performance metrics of VGG16 based model

In [6]:
print("VALIDATION ACCURACIES\n")
print_performance(valid_perf)


VALIDATION ACCURACIES


acc =  94.61
              sensitivity  specificity      precision      f1-score

Class Normal:  98.17 ,        96.35 ,        96.04 ,        97.08

Class Early:   69.53 ,        98.45 ,        81.54 ,        73.83

Class Late:    95.31 ,        96.3 ,        95.54 ,        95.38

Average

sensitivity:  87.67 , specificity:  97.04 , precision:  91.04 , f1-score:  88.76


### 8. Performance of model on test set

In [8]:
perf = {}

for i in range(1,6):
    model = build_model_intra(i)      
    ans = model.predict(t_img)

    # performance metrics  
    ans = np.argmax(ans, axis =1)
    perf[i]=performance_metrics(t_y,ans)


print("TEST ACCURACIES\n")
print_performance(perf)


TEST ACCURACIES


acc =  90.14
              sensitivity  specificity      precision      f1-score

Class Normal:  99.92 ,        94.16 ,        85.23 ,        91.95

Class Early:   66.64 ,        99.68 ,        98.73 ,        78.99

Class Late:    97.0 ,        89.52 ,        90.45 ,        93.55

Average

sensitivity:  87.85 , specificity:  94.45 , precision:  91.47 , f1-score:  88.16
